In [2]:
import numpy as np
import pandas as pd
import random
import math

In [27]:
def getdistancematrix(lat,long):
    n = len(lat)
    distancematrix = np.zeros([n,n])
    for i in range(0,n):
        for j in range(0,n):
            distancematrix[i,j] = 1.3*69*math.sqrt(math.pow(lat[i]-lat[j],2)+math.pow(long[i]-long[j],2))

    return distancematrix

In [31]:
C = 25
data = pd.read_csv('processed_data.csv')
demand = data['machine'].values
tw = data[['start','end']].values
long = data['long'].values
lat = data['lat'].values

dismatrix = getdistancematrix(lat,long)
timematrix = np.round(2.4 * dismatrix,2)
dismatrix = np.round(dismatrix,2)

array([[ 0.  ,  4.99, 25.73, ..., 12.77, 27.4 ,  0.  ],
       [ 4.99,  0.  , 27.8 , ...,  9.93, 27.79,  4.99],
       [25.73, 27.8 ,  0.  , ..., 37.69,  9.5 , 25.73],
       ...,
       [12.77,  9.93, 37.69, ...,  0.  , 37.51, 12.77],
       [27.4 , 27.79,  9.5 , ..., 37.51,  0.  , 27.4 ],
       [ 0.  ,  4.99, 25.73, ..., 12.77, 27.4 ,  0.  ]])

In [35]:
# calculate the distance
def getdistance(path,dismatrix):
    n = len(path)
    distance = 0
    for i in range(0,n-1):
        distance += dismatrix[path[i],path[i+1]]
    return distance

# calculate invalid time
def getinvalidtime(path,timematrix,tw):
    n = len(path)
    invalidtime = 0
    ul = np.zeros(n)
    for i in range(0,n):
        invalidtime_i = 0
        if path[i] ==0:
            continue
        else:
            reachtime = ul[i-1] + timematrix[path[i-1],path[i]] + 30
            if reachtime <= tw[i,0]:
                ul[i] = tw[i,0]
            elif tw[i,0]<reachtime<=tw[i,1]: 
                ul[i] = reachtime
            else:
                ul[i] = reachtime
                invalidtime_i += reachtime - tw[i,1]
            finishtime = reachtime+30
            if finishtime > tw[i,1]:
                invalidtime_i += finishtime-tw[i,1]
        invalidtime += invalidtime_i
    
    return invalidtime

# calculate invalid vehicle
def getinvalidvehicle (path,demand,C):
    invalidcap = 0
    n = len(path)
    acumcap = np.zeros(n)
    for i in range(0,n):
        if path[i] == 0:
            continue
        else:
            acumcap[i] = acumcap[i-1] + demand[i]
    
    end = np.where(acumcap)[0]
    end = np.delete(end,0)
    for id in end:
        if acumcap[id-1]>C:
            invalidcap += C - acumcap[id-1]
    return invalidcap

def getobjective(path,distancecost,timepenalty,capacitypenalty,dismatrix,timematrix,tw,demand,C):
    objective = distancecost*getdistance(path,dismatrix) + timepenalty*getinvalidtime(path,timematrix,tw)+capacitypenalty*getinvalidvehicle(path,demand,C)
    return objective



In [40]:
path = [0,1,0,5,6,0]
# getdistance(path,dismatrix)
# getinvalidvehicle (path,demand,C)
getinvalidtime(path,timematrix,tw)

0

In [194]:
# solution generation
path = [0,1,3,4,0,8,9,0]
def swap(path):
    n = len(path)
    point = random.sample(range(1,n-1),2)
    a = point[0]
    b = point[1]
    path[a],path[b] = path[b],path[a]
    return path

def insert(path):
    n = len(path)
    choose = random.sample(range(1,n-1),1)[0]
    location = path[choose]
    path.pop(choose)
    path.insert(random.randint(1,n-2),location)

    return path

def cross(path):
    n = len(path)
    choose = sorted(random.sample(range(1,n-1),2))
    a = choose[0]
    b = choose[1]
    route = path[a:b]
    route.reverse()
    path[a:b] = route
    
    return path

[0, 1, 3, 4, 0, 8, 9, 0]
[0, 8, 0, 4, 3, 1, 9, 0]


In [16]:
def choosemethod(successlist):
    p = successlist/np.sum(successlist)
    choice = np.random.choice([0, 1, 2], p = p.ravel())

    return choice

1

In [25]:
# main
T = 500
alpha = 0.99974
FinalT = 0.001
timepenalty = 300
capacitypenalty = 500 
distancecost = 2.9
successlist = np.ones(3)
oldpath = [0]

while T > FinalT:
    
    methodchoice = choosemethod(successlist)
    if methodchoice==0:
        newpath = swap(oldpath)
    elif methodchoice==1:
        newpath = insert(oldpath)
    else:
        newpath = cross(oldpath)
    oldobjective = getobjective(oldpath,distancecost,timepenalty,capacitypenalty,dismatrix,timematrix,tw,demand,C)
    newobjective = getobjective(newpath,distancecost,timepenalty,capacitypenalty,dismatrix,timematrix,tw,demand,C)
    d_value = newobjective - oldobjective
    if d_value <=0:
        # print('accept')
        oldpath = newpath.copy()
        successlist[methodchoice] +=1
    else:
        p = math.exp(-d_value / T)
        if random.random() <=p:
            #print('bad but accept')
            oldpath = newpath.copy()
    T = T* alpha

array([1., 1., 1.])